<a href="https://colab.research.google.com/github/Chumunaca/DuranIa/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ESTO SE ENTRENO CON CNN1 Y YOLO (VER PARA VER DE DONDE SALIERON LAS COSAS), SE JUNTARON LOS 3 CSV DE VINDR EN UNIFICACIONVINDR (VER)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q streamlit pyngrok ultralytics timm joblib scikit-learn opencv-python-headless

In [ ]:
%%writefile app.py
import streamlit as st
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from PIL import Image
import torchvision.transforms as transforms
import timm
import joblib
from ultralytics import YOLO
import cv2
import os
import tempfile

# --- Configuración de la página ---
st.set_page_config(
    page_title="Sistema de trIAge Mamográfico",
    page_icon="🧠",
    layout="wide"
)

# --- Arquitecturas del modelo ---
class MetadataANN(nn.Module):
    def __init__(self, input_dim, output_dim=128):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        return self.relu(self.fc3(x))

class MultiViewHybridModel(nn.Module):
    def __init__(self, num_meta_features, num_classes):
        super().__init__()
        self.cnn_base = timm.create_model('efficientnet_b4', pretrained=False)
        num_cnn_features = self.cnn_base.classifier.in_features
        self.cnn_base.classifier = nn.Identity()
        self.ann_arm = MetadataANN(num_meta_features, output_dim=128)
        num_ann_features = 128
        self.fusion_head = nn.Sequential(
            nn.Linear(num_cnn_features + num_ann_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, imgs, meta):
        batch_size = imgs.size(0)
        num_views = imgs.size(1)
        imgs_flat = imgs.view(batch_size * num_views, 3, 224, 224)
        cnn_features_flat = self.cnn_base(imgs_flat)
        cnn_features = cnn_features_flat.view(batch_size, num_views, -1)
        img_features_pooled = torch.mean(cnn_features, dim=1)
        meta_features = self.ann_arm(meta)
        combined = torch.cat((img_features_pooled, meta_features), dim=1)
        return self.fusion_head(combined)

# --- Funciones de carga y predicción ---
@st.cache_resource
def load_all_models():
    birads_path = '/content/drive/MyDrive/TESIS/checkpoints_birads_v2.1/best_model_birads_v2.1.pth'
    yolo_path = '/content/drive/MyDrive/TESIS/yolov8_models/best.pt'
    preprocessor_path = '/content/drive/MyDrive/TESIS/preprocessor.pkl'
    label_encoder_path = '/content/drive/MyDrive/TESIS/label_encoder.pkl'

    paths = {
        "Cerebro (BI-RADS)": birads_path,
        "Ojos (YOLO)": yolo_path,
        "Preprocesador": preprocessor_path,
        "Label Encoder": label_encoder_path
    }

    missing_files = [name for name, path in paths.items() if not os.path.exists(path)]
    if missing_files:
        for name in missing_files:
            st.error(f"¡ERROR CRÍTICO! No se encontró el archivo para '{name}'. Verifica la ruta en tu Drive.")
        return None, None, None, None

    preprocessor = joblib.load(preprocessor_path)
    label_encoder = joblib.load(label_encoder_path)
    # Crear un DataFrame dummy CON LOS TIPOS DE DATOS CORRECTOS
    dummy_data = {}
    for col in preprocessor.feature_names_in_:
        if col == 'Age':
            dummy_data[col] = [50]  # Un int de ejemplo
        elif col == 'breast_density':
            dummy_data[col] = ['DENSITY A'] # Un STRING de categoría válido
        else:
            # Asumir que el resto (como findings) son numéricos
            dummy_data[col] = [0]

    # Crear el DF asegurando el orden de columnas que el preprocesador aprendió
    dummy_df = pd.DataFrame(dummy_data, columns=preprocessor.feature_names_in_)

    # Ahora sí, transforma el dummy_df que está bien hecho
    num_meta_features = preprocessor.transform(dummy_df).shape[1]


    num_classes = len(label_encoder.classes_)

    cerebro_model = MultiViewHybridModel(num_meta_features, num_classes)
    cerebro_model.load_state_dict(torch.load(birads_path, map_location=torch.device('cpu')))
    cerebro_model.eval()

    ojos_model = YOLO(yolo_path)

    st.success("¡Todos los modelos y preprocesadores cargados!")
    return cerebro_model, ojos_model, preprocessor, label_encoder

def predict_birads(cerebro_model, preprocessor, label_encoder, images, age, density, finding_cols):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.485, 0.485], std=[0.229, 0.229, 0.229])
    ])

    processed_images = [transform(Image.open(img).convert('RGB')) for img in images]
    while len(processed_images) < 4:
        processed_images.append(torch.zeros((3, 224, 224)))

    imgs_tensor = torch.stack(processed_images).unsqueeze(0)

    meta_df_data = {'Age': [age], 'breast_density': [density]}
    for col in finding_cols:
        meta_df_data[col] = [0]

    meta_df = pd.DataFrame(meta_df_data)
    # Asegurar que el DataFrame tenga las columnas en el orden correcto que espera el preprocesador
    meta_df = meta_df.reindex(columns=preprocessor.feature_names_in_, fill_value=0)

    meta_pre = preprocessor.transform(meta_df)
    meta_tensor = torch.tensor(meta_pre, dtype=torch.float32)

    with torch.no_grad():
        outputs = cerebro_model(imgs_tensor, meta_tensor)
        probs = torch.softmax(outputs, dim=1).squeeze().numpy()
        pred_idx = np.argmax(probs)
        pred_class = label_encoder.inverse_transform([pred_idx])[0]
        confianza = probs[pred_idx]

    return pred_class, confianza, probs

def predict_yolo_and_draw(ojos_model, images):
    results_with_boxes = []
    with tempfile.TemporaryDirectory() as temp_dir:
        for i, img_file in enumerate(images):
            # Guardar temporalmente el archivo subido para que cv2.imread pueda leerlo
            temp_path = os.path.join(temp_dir, f"image_{i}.png")
            with open(temp_path, "wb") as f:
                f.write(img_file.getbuffer())

            img = cv2.imread(temp_path)
            if img is None: # Chequeo por si acaso cv2 no puede leer la imagen
                st.warning(f"No se pudo leer la imagen {i+1}. Omitiendo.")
                continue

            results = ojos_model(img, verbose=False)

            detections = []
            for r in results:
                for box in r.boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    conf, cls = box.conf[0].item(), int(box.cls[0].item())
                    label = ojos_model.names[cls]
                    color = (0, 255, 0)
                    cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(img, f'{label} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                    detections.append(f"- {label} ({conf:.2f})")

            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results_with_boxes.append((img_rgb, detections))

    return results_with_boxes

# --- Interfaz de la aplicación ---
st.title("🤖 Sistema de trIAge Mamográfico (Cerebro + Ojos)")
st.caption("Prototipo V2.1 - Sube las 4 imágenes del estudio y los metadatos para análisis.")
st.caption("Esto solo es un prototiopo, aún no se puede usar para implementación clínica")
cerebro, ojos, preprocessor, label_encoder = load_all_models()

if cerebro:
    # Asegurarnos de obtener las columnas 'finding_' que el preprocesador realmente conoce
    finding_columns_expected = [c for c in preprocessor.feature_names_in_ if c.startswith('finding_')]

    col1, col2 = st.columns([1, 2])

    with col1:
        st.header("📂 Carga de Datos")

        uploaded_files = st.file_uploader(
            "Sube 1 a 4 imágenes del estudio",
            type=['png', 'jpg', 'jpeg'],
            accept_multiple_files=True
        )

        age = st.number_input("Edad de la Paciente", min_value=18, max_value=100, value=50)
        density = st.selectbox("Densidad Mamaria", options=['DENSITY A', 'DENSITY B', 'DENSITY C', 'DENSITY D'])

        analyze_button = st.button("🧠 Analizar Estudio", type="primary", use_container_width=True)

    with col2:
        st.header("📊 Resultados del Análisis")

        if analyze_button and uploaded_files:
            # Validar que no sean más de 4 imágenes
            if len(uploaded_files) > 4:
                st.error("¡Demasiadas imágenes! Por favor, sube un máximo de 4.")
            else:
                with st.spinner("El 'Cerebro' está pensando..."):
                    pred_birads, conf_birads, all_probs = predict_birads(
                        cerebro, preprocessor, label_encoder, uploaded_files, age, density, finding_columns_expected
                    )

                    st.subheader("Veredicto del Cerebro (trIAge):")
                    if "4" in str(pred_birads) or "5" in str(pred_birads):
                        st.error(f"**Predicción: {pred_birads}** (Confianza: {conf_birads:.2%})")
                        st.warning("⚠️ ¡¡¡ALERTA!!! ESTUDIO DE ALTA PRIORIDAD.")
                    elif "3" in str(pred_birads):
                        st.warning(f"**Predicción: {pred_birads}** (Confianza: {conf_birads:.2%})")
                        st.info("ℹ️ Estudio de prioridad media. Requiere seguimiento.")
                    else:
                        st.success(f"**Predicción: {pred_birads}** (Confianza: {conf_birads:.2%})")
                        st.info("✅ Estudio de baja prioridad.")

                    st.write("Probabilidades por clase:")
                    prob_df = pd.DataFrame(all_probs, index=label_encoder.classes_, columns=['Probabilidad'])
                    st.dataframe(prob_df.style.format("{:.2%}"))

                # Ejecutar YOLO solo si la predicción es 3, 4 o 5
                if "3" in str(pred_birads) or "4" in str(pred_birads) or "5" in str(pred_birads):
                    st.subheader("Análisis de los Ojos (Detección de Lesiones):")
                    with st.spinner("Los 'Ojos' están buscando..."):
                        results_yolo = predict_yolo_and_draw(ojos, uploaded_files)

                        if results_yolo:
                            img_cols = st.columns(len(results_yolo))
                            for i, (img_with_box, detections) in enumerate(results_yolo):
                                with img_cols[i]:
                                    st.image(img_with_box, caption=f"Imagen {i+1}")
                                    if detections:
                                        for d in detections:
                                            st.write(d)
                                    else:
                                        st.write("- Sin hallazgos.")
                        else:
                            st.info("Los 'Ojos' no procesaron ninguna imagen (posiblemente error de lectura).")

        elif analyze_button:
            st.error("Por favor, sube al menos una imagen para analizar.")
else:
    st.header("La aplicación no puede iniciar. Faltan archivos de modelo.")

In [ ]:
from pyngrok import ngrok
import os
import time
import subprocess

# configurar ngrok
NGROK_TOKEN = "34NS6us1Z7pxsVXGAcbwHc1j3EM_2moHHqTx5g215GF3inFm5"
!ngrok config add-authtoken {NGROK_TOKEN}

# limpiar antiguos procesos
ngrok.kill()
os.system("killall -9 ngrok")
os.system("killall -9 streamlit")
time.sleep(2)

# lanzador
print("Launching Streamlit...")
streamlit_proc = subprocess.Popen(["streamlit", "run", "app.py", "--server.port=8501"])

# ngrok tunel
public_url = ngrok.connect(8501)
print(f"App running at: {public_url}")

# mantener celda corriendo
try:
    streamlit_proc.wait()
except KeyboardInterrupt:
    print("Stopping...")
    streamlit_proc.terminate()
    ngrok.kill()